Spark Assignment #4 : Spark Stream on Twitter Data

Author: Tryambak Kaushik

Q: Complete the Twitter Hashtag example using Twitter Developer Account

In [1]:
#!pip install requests_oauthlib

In [2]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [3]:
# Replace the values below with yours
ACCESS_TOKEN = '1216192078230757376-zpl6aGno1cD5yHYGiDKWlRgb1jGS'
ACCESS_SECRET = 'jMlsXL8NPnjc5qhGcFFtFjGQz6A5wWJeaPdUgIWdVJt'
CONSUMER_KEY = 'ShZsbBoOSfRKs8BAKIJY9Up'
CONSUMER_SECRET = 'VP8xFzieqpPpRL6ny9oudLwCQNd9XHH7RnThiVjoNk7ROtTw'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In [4]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
#     print(query_url, response)
    return response

In [5]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            print("Tweet Text: " + tweet_text)
            print ("------------------------------------------")
            tcp_connection.send(tweet_text + '\n')
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

In [6]:
TCP_IP = "localhost"
TCP_PORT = 9997
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(5)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

Tweet Text: swore he was from panama but ok
------------------------------------------
Tweet Text: @JamesFrancoNews 
Awesome page turner!💪☝👍👌💯
#book #selfimprovement #shortstories #HealthForAll
https://t.co/F4MfsZbkDm
------------------------------------------
Error: <type 'exceptions.UnicodeEncodeError'>
Tweet Text: @suevee85 I admire your optimism but he’s not gonna do any of that
------------------------------------------
Error: <type 'exceptions.UnicodeEncodeError'>
Tweet Text: I miss doing stuff. https://t.co/7ByQEJhjVa
------------------------------------------
Tweet Text: If i drive you around all the time and you refuse to do the same w me in ur car... don’t ask to chill 🥴
------------------------------------------
Error: <type 'exceptions.UnicodeEncodeError'>


In [7]:
# socket.close()

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [4]:
spc = SparkContext("local[2]", "LocalWordCount")
stc = StreamingContext(spc, 5)
stc.checkpoint("checkpoint")

In [5]:
def get_hashtag(line):
    hastag_name = line.strip()

    if hastag_name == '#hadoop':
        output = 'Hadoop'
    elif hastag_name == '#spark':
        output = 'Spark'
    elif hastag_name == '#bigdata':
        output = 'Big Data'
    else:
        output = 'Unknown'

    return (output, 1)

In [6]:
batch_interval = 1 
window_length = 30
frequency = 10

lines = stc.socketTextStream("localhost", 9997)

addFunc = lambda x, y: x + y
invAddFunc = lambda x, y: x - y
window_counts = lines.map(get_hashtag).reduceByKeyAndWindow(addFunc, invAddFunc,window_length, frequency)

In [7]:
window_counts.pprint()

In [8]:
stc.start()             # Start the computation
stc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2020-06-18 20:04:25
-------------------------------------------
('Unknown', 26)

-------------------------------------------
Time: 2020-06-18 20:04:35
-------------------------------------------
('Unknown', 95)

-------------------------------------------
('Unknown', 195)

-------------------------------------------
Time: 2020-06-18 20:06:15
-------------------------------------------
('Unknown', 217)

